## Imports

In [1]:
%matplotlib inline

import os
from scipy import ndimage as ndi, signal
from skimage import io, filters, exposure, color, img_as_float
import matplotlib.pylab as plt
import matplotlib
import numpy as np
from ipywidgets import interact, widgets, HBox
from IPython import display
import math
from random import randint

#Ajustar el tamaño de las imágenes:
matplotlib.rcParams['figure.figsize'] = (14,12)

## Gabor local (Incompleto)

In [2]:
def gaborFilterCoord(x, y, orientation, bandwidth, sigmaX, sigmaY):
    thetaX = x * math.cos(orientation) - y * math.sin(orientation)
    thetaY = x * math.sin(orientation) - y * math.cos(orientation)
    
    firstTerm = math.exp ( -0.5 * ((thetaX ** 2 / sigmaX ** 2) + (thetaY ** 2 / sigmaY ** 2)) )
    secondTerm = math.cos(2 * math.pi / bandwidth * thetaX )
    
    return firstTerm * secondTerm;

gaborFilterCoord(1, 3, 0, 20, 10, 10)

0.9046729426630928

## Gabor de Wikipedia

In [3]:
#x,y: es una coordenada en el filtro. Si definimos un filtro con radios rx, ry,
#entonces x ∈ [−rx,rx] e y ∈ [−ry,ry]. Recuerde, además, que el tamaño
#de un filtro gaussiano depende de su desviación estándar correspondiente.

def gabor_fn(theta, Lambda, sigma_x, sigma_y):

    # Bounding box
    nstds = 3 # Number of standard deviation sigma
    xmax = max(abs(nstds * sigma_x * np.cos(theta)), abs(nstds * sigma_y * np.sin(theta)))
    xmax = np.ceil(max(1, xmax))
    ymax = max(abs(nstds * sigma_x * np.sin(theta)), abs(nstds * sigma_y * np.cos(theta)))
    ymax = np.ceil(max(1, ymax))
    xmin = -xmax
    ymin = -ymax
    (y, x) = np.meshgrid(np.arange(ymin, ymax + 1), np.arange(xmin, xmax + 1))

    # Rotation 
    x_theta = x * np.cos(theta) + y * np.sin(theta)
    y_theta = -x * np.sin(theta) + y * np.cos(theta)

    gb = np.exp(-.5 * (x_theta ** 2 / sigma_x ** 2 + y_theta ** 2 / sigma_y ** 2)) * np.cos(2 * np.pi / Lambda * x_theta)
    return gb

## Prueba Gabor

In [4]:
gabor = gabor_fn(0, 10, 10, 10)
print(gabor.shape)
#gabor

(61, 61)


## Gabor dinámico: Prueba de parámetros

In [5]:
def testGabor(theta = math.pi/4, Lambda = 30, sigma_x=40, sigma_y=40):
    gabor = gabor_fn(theta, Lambda, sigma_x, sigma_y)
    #fig, ax = plt.subplots()
    plt.imshow(gabor, cmap="binary")
    plt.show()
    
interact(testGabor, theta = (0, math.pi), Lambda = (0,100), sigma_x=(0,100), sigma_y=(0,100))

<function __main__.testGabor>

## Celdas de pixeles

In [6]:
W = 4
for i in range(W*W):
    print(math.floor(i/W))

0
0
0
0
1
1
1
1
2
2
2
2
3
3
3
3


## Demo Celdas: Celdas por imagen

In [ ]:
def createCellDemo(imageSize, cellSize):
    cellImage = np.zeros((imageSize, imageSize))
    cellCount = math.floor(imageSize/cellSize)
    cellPixels = np.zeros((cellSize, cellSize, cellCount))
    print(cellCount)
    
createCellDemo(64,4) 

16


## Demo Celdas: Índices celdas

In [ ]:
def createCellDemo(imageSize, cellSize):
    cellImage = np.zeros((imageSize, imageSize))
    cellCount = math.floor(imageSize/cellSize)
    cellPixels = np.zeros((cellCount, cellCount)) 
    x = 0
    for j in range(imageSize):
        for i in range(imageSize):
            cellX = math.floor(i/cellSize)
            cellY = math.floor(j/cellSize)
            cellIndex = cellY * cellCount + cellX
            if x < 16:
                print("i:",i,"j:", j,"cellX:", cellX,"cellY:", cellY,"index:", cellIndex)
                x += 1
    
createCellDemo(16,4)

## Demo Celdas: Visualización celdas

In [ ]:
def createCellDemo(imageSize, cellSize):
    cellImage = np.zeros((imageSize, imageSize))
    cellCount = math.floor(imageSize/cellSize)
    cellPixels = np.zeros((cellCount, cellCount)) 
    for j in range(imageSize):
        for i in range(imageSize):
            cellX = math.floor(i/cellSize)
            cellY = math.floor(j/cellSize)
            cellIndex = cellX * cellCount + cellY
            
            cellPixels[cellX, cellY] = cellIndex
    
    
    plt.imshow(cellPixels, cmap="gray")
        
createCellDemo(16,4)

In [ ]:
createCellDemo(32,4)

## Demo Celdas: Anotación en celdas

In [ ]:
def createCellDemo(imageSize, cellSize):
    cellImage = np.zeros((imageSize, imageSize))
    cellCount = math.floor(imageSize/cellSize)
    cellPixels = np.zeros((cellCount, cellCount)) 
    for j in range(imageSize):
        for i in range(imageSize):
            cellX = math.floor(i/cellSize)
            cellY = math.floor(j/cellSize)
            cellIndex = cellX * cellCount + cellY
            
            cellPixels[cellX, cellY] = cellIndex
    
    
    fig, ax = plt.subplots()
    ax.imshow(cellPixels)
    
    for j in range(cellPixels.shape[1]):
        for i in range(cellPixels.shape[0]):
            ax.annotate('{0:.0f}'.format(cellPixels[i,j]), (j,i))
        
createCellDemo(32,4)

In [ ]:
createCellDemo(64,4)

## Demo Celdas: Parametrizar anotación

In [ ]:
def decorateCells(ax, cellPixels, cellContent = None):
    if cellContent == "index":
        for j in range(cellPixels.shape[1]):
            for i in range(cellPixels.shape[0]):
                ax.annotate(cellPixels[i,j], (j,i))
                
    return ax, cellPixels

def createCellDemo(imageSize, cellSize, cellContent = None):
    cellImage = np.zeros((imageSize, imageSize))
    cellCount = math.floor(imageSize/cellSize)
    cellPixels = np.zeros((cellCount, cellCount)) 
    for j in range(imageSize):
        for i in range(imageSize):
            cellX = math.floor(i/cellSize)
            cellY = math.floor(j/cellSize)
            cellIndex = cellX * cellCount + cellY
            
            if cellContent == "index" or cellContent == None:
                cellPixels[cellX, cellY] = cellIndex
    
    fig, ax = plt.subplots()
    ax, cellPixels = decorateCells(ax, cellPixels, cellContent)
    ax.imshow(cellPixels)
    
createCellDemo(32,4)

## Demo Celdas: Visualización de orientaciones

In [ ]:
def decorateCells(fig, ax, cells, cellContent = None):
    if cellContent == "index":
        for j in range(cells.shape[1]):
            for i in range(cells.shape[0]):
                ax.annotate(cells[i,j], (j,i))
                
    if cellContent == "orientations":
        for j in range(cells.shape[1]):
            for i in range(cells.shape[0]):
                ax.plot([j-math.cos(cells[i,j])/4,j+math.cos(cells[i,j])/4],[i+math.sin(cells[i,j])/4,i-math.sin(cells[i,j])/4])
                ax.annotate('{:1.2f}'.format(cells[i,j]), (j-1/3,i-1/3))
                cells[i,j] = 0;

    return fig, ax, cells

def createCellDemo(imageSize, cellSize, cellContent = None):
    cellImage = np.zeros((imageSize, imageSize))
    cellCount = math.floor(imageSize/cellSize)
    cells = np.zeros((cellCount, cellCount)) 
    for j in range(imageSize):
        for i in range(imageSize):
            cellX = math.floor(i/cellSize)
            cellY = math.floor(j/cellSize)
            cellIndex = cellX * cellCount + cellY
            
            if cellContent == "index" or cellContent == None:
                cells[cellX, cellY] = cellIndex
            elif cellContent == "orientations":
                cells[cellX, cellY] = math.pi*cellIndex/cellCount/2
                
    fig, ax = plt.subplots()
    fig, ax, cells = decorateCells(fig, ax, cells, cellContent)
    ax.imshow(cells, cmap="binary")
    
createCellDemo(32,4, "orientations")

## Orientación: Celdas según intesidades

In [ ]:
def imageOrientation(image, cellSize):
    rows, cols = image.shape
    cellRows = math.floor(rows/cellSize)
    cellCols = math.floor(cols/cellSize)
    cellCount = cellRows*cellCols
    cells = np.zeros((cellRows, cellCols)) 
    
    for j in range(cols):
        for i in range(rows):
            cellX = math.floor(i/cellSize)
            cellY = math.floor(j/cellSize)
            pixelValue = image[i,j]
            if(pixelValue<255):
                cells[cellX, cellY] = pixelValue
            
    fig, ax = plt.subplots()
    ax.imshow(cells, cmap="binary")

image = io.imread("./DB1_B/101_2.tif")
imageOrientation(image, 6)            

## Orientación: Orientación de intensidades

In [ ]:
def imageOrientation(image, cellSize):
    rows, cols = image.shape
    cellRows = math.floor(rows/cellSize)
    cellCols = math.floor(cols/cellSize)
    cellCount = cellRows*cellCols
    cells = np.zeros((cellRows, cellCols)) 
    
    for j in range(cols):
        for i in range(rows):
            cellX = math.floor(i/cellSize)
            cellY = math.floor(j/cellSize)
            pixelValue = image[i,j]
            if(pixelValue<255):
                cells[cellX, cellY] = 0.5 * math.pi * pixelValue / cellCount
            
    fig, ax = plt.subplots()
    for j in range(cellCols):
        for i in range(cellRows):
            if(cells[i,j]>0):
                ax.plot([j-math.cos(cells[i,j])/2,j+math.cos(cells[i,j])/2],[i-math.sin(cells[i,j])/2,i+math.sin(cells[i,j])/2])
            cells[i,j] = 0;
                
    ax.imshow(cells, cmap="binary")

imageFile = "./DB1_B/101_2.tif"
image = io.imread(imageFile)
imageOrientation(image, 12)    

## Orientación: Algoritmo de orientaciones

In [ ]:
def imageOrientation(image, cellSize):
    rows, cols = image.shape
    cellRows = math.floor(rows/cellSize)
    cellCols = math.floor(cols/cellSize)
    cellCount = cellRows*cellCols

    sobelH = filters.sobel_h(image)
    sobelV = filters.sobel_v(image)
    
    cells = np.zeros((cellRows, cellCols)) 
    m_sines = np.zeros((cellRows, cellCols)) 
    m_cosines = np.zeros((cellRows, cellCols)) 
    
    for j in range(cols):
        for i in range(rows):
            if(image[i,j] == 255):
                continue
            cellX = math.floor(i/cellSize)
            cellY = math.floor(j/cellSize)
            gx = sobelH[i,j]
            gy = sobelV[i,j]
            alfa = math.atan2(gy, gx)
            mag = math.sqrt(gx * gx + gy * gy)
            m_sines[cellX, cellY] = m_sines[cellX, cellY] + 2 * math.cos(alfa) * math.sin(alfa) * mag
            m_cosines[cellX, cellY] = m_cosines[cellX, cellY] + (math.cos(alfa) * math.cos(alfa) - math.sin(alfa) * math.sin(alfa)) * mag
            cellValue = math.pi + 0.5*math.atan2(m_sines[cellX, cellY], m_cosines[cellX, cellY])
            cells[cellX, cellY] = cellValue
            
    fig, ax = plt.subplots()
    for j in range(cellCols):
        for i in range(cellRows):
            if(cells[i,j]> 0):
                ax.plot([j - 0.5 * math.cos(cells[i,j]), j + 0.5 * math.cos(cells[i,j])],[i + 0.5 * math.sin(cells[i,j]), i - 0.5 * math.sin(cells[i,j])], color="black")
            cells[i,j] = 0;
                
    ax.imshow(cells, cmap="binary")

imageFile = "./DB1_B/101_2.tif"
image = io.imread(imageFile)
imageOrientation(image, 8)   

## Orientación: Pruebas sobre celdas

In [ ]:
def segmentImagen(image, cellSize):
    rows, cols = image.shape
    cellRows = math.floor(rows/cellSize)
    cellCols = math.floor(cols/cellSize)
    cellCount = cellRows*cellCols
    cells = np.zeros((cellRows, cellCols)) 
    
    for j in range(cols):
        for i in range(rows):
            if(image[i,j] == 255):
                continue
            cellX = math.floor(i/cellSize)
            cellY = math.floor(j/cellSize)
            if(cellX % 2==0 and cellY % 2 ==0):
                cells[cellX, cellY] = image[i,j]
    
    fig, ax = plt.subplots()
    ax.imshow(cells, cmap="binary")
    
imageFile = "./DB1_B/101_2.tif"
image = io.imread(imageFile)
segmentImagen(image, 8)      

## Orientación: Gabor sobre orientaciones

In [ ]:
def imageOrientation(image, cellSize):
    """Mostrar Imagen original"""
    if(False):
        fig, ax = plt.subplots()
        ax.imshow(image, cmap="gray")
    
    """Init"""
    rows, cols = image.shape
    cellRows = math.floor(rows/cellSize)
    cellCols = math.floor(cols/cellSize)
    cellCount = cellRows*cellCols
    
    cells = np.zeros((cellRows, cellCols)) 
    m_sines = np.zeros((cellRows, cellCols)) 
    m_cosines = np.zeros((cellRows, cellCols)) 
    
    """Calcular orientaciones"""
    sobelH = filters.sobel_h(image)
    sobelV = filters.sobel_v(image)
    
    for j in range(cols):
        for i in range(rows):
            #if(image[i,j] == 255):
            #    continue
            cellX = math.floor(i/cellSize)
            cellY = math.floor(j/cellSize)
            gx = sobelH[i,j]
            gy = sobelV[i,j]
            alfa = math.atan2(gy, gx)
            mag = math.sqrt(gx * gx + gy * gy)
            m_sines[cellX, cellY] = m_sines[cellX, cellY] + 2 * math.cos(alfa) * math.sin(alfa) * mag
            m_cosines[cellX, cellY] = m_cosines[cellX, cellY] + (math.cos(alfa) * math.cos(alfa) - math.sin(alfa) * math.sin(alfa)) * mag
            orientation = math.pi + 0.5 * math.atan2(m_sines[cellX, cellY], m_cosines[cellX, cellY])
            cells[cellX, cellY] = orientation
    
    """Mostrar orientaciones"""
    if(False):
        fig, ax = plt.subplots()
        arrowCells = np.zeros_like(cells)
        for j in range(cellCols):
            for i in range(cellRows):
                if(cells[i,j]> 0):
                    ax.plot([j - 0.5 * math.cos(cells[i,j]), j + 0.5 * math.cos(cells[i,j])],[i + 0.5 * math.sin(cells[i,j]), i - 0.5 * math.sin(cells[i,j])], color="black")

        ax.imshow(arrowCells, cmap="binary")
    
    """Mejorar imagen"""
    improvedImage = np.zeros((rows, cols))
    
    for j in range(cellCols):
        for i in range(cellRows):
            orientation = cells[i, j]
            filtered = signal.correlate2d(image[(cellSize * i) : (cellSize) * (i + 1), (cellSize * j) : (cellSize) * (j + 1)], gabor_fn(orientation, 10, 10, 10), mode="same")
            improvedImage[(cellSize * i) : (cellSize) * (i + 1), (cellSize * j) : (cellSize) * (j + 1)] = filtered
                
    fig, ax = plt.subplots()
    ax.imshow(improvedImage, cmap="gray")
    
imageFile = "./DB1_B/101_2.tif"
image = io.imread(imageFile)
imageOrientation(image, 8)   